# Labeled Point Data

In this notebook we're going to implement a custom tiles handler

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data/spatial_gene_expression.csv.gz')

print('Number of points: {}'.format(len(df)))
df.head()

Number of points: 2601157


,x,y,gene
0,791.8022,229.1912,ENO1
1,901.6493,897.6785,MTHFD1
2,708.8124,902.3960,THBS1
3,-690.2937,-870.6509,PPM1G
4,-52.4658,-278.9252,ENO1


In [4]:
from higlass.tilesets import Tileset

def dfdensity(df, channel_col: str = None, channel: str = None, uuid: str = None, max_zoom: int = 30):
    from clodius.tiles.format import format_dense_tile
    from clodius.tiles.utils import bundled_tiles_wrapper_2d, tile_bounds
    
    # Tileset info
    info = {
        'min_pos': [df['x'].min(), df['y'].min()],
        'max_pos': [df['x'].max(), df['y'].max()],
        'max_width': max(
            df['x'].max() - df['x'].min(),
            df['y'].max() - df['y'].min()
        ),
        'max_zoom': max_zoom,
        'mirror_tiles': False
    }
    
    # Subsample data
    if channel_col and channel:
        data = df[df[channel_col] == channel].reindex(columns=['x', 'y']).values
    else:
        data = df.reindex(columns=['x', 'y']).values
    
    
    def tileset_info():
        return info
    
    
    def filter_points(out, extent):
        out = out[out[:, 0] > extent[0]]
        out = out[out[:, 0] < extent[2]]
        out = out[out[:, 1] > extent[1]]
        out = out[out[:, 1] < extent[3]]

        return out


    def density_tiles(z, x, y, width=1, height=1):
        returns = []
        
        out = data.copy()

        # get all the points in the region
        all_points = filter_points(out, tile_bounds(info, z, x, y, width, height))

        for i in range(width):
            for j in range(height):
                # filter from the larger subregion
                filtered_points = filter_points(all_points, tile_bounds(info, z, x + i, y + j))

                hist = np.histogram2d(
                    filtered_points[:, 0],
                    filtered_points[:, 1],
                    bins=256
                )[0].T
                hist[hist == 0.] = np.nan

                returns += [((z, x + i, y + j), hist)]

        return returns
    
    def get_tile(z, x, y, width: int = 1, height: int = 1):
        return [(tile_position, format_dense_tile(d.flatten())) for
                (tile_position, d) in density_tiles(z, x, y, width, height)]
    
    def tiles(tile_ids):
        return bundled_tiles_wrapper_2d(tile_ids, get_tile)

    return Tileset(
        uuid=uuid,
        tileset_info=tileset_info,
        tiles=tiles
    )

In [5]:
ts_density = dfdensity(df, uuid='density')

In [6]:
import higlass
from higlass.client import Track, View

display, server, viewconf = higlass.display([
    View([
        Track('left-axis'),
        Track('top-axis'),
        Track('heatmap',
            tileset=ts_density,
            position='center',
            height=600,
            options={
                'colorRange': ['rgba(245,166,35,1.0)', 'rgba(208,2,27,1.0)', 'black'],
                'backgroundColor': 'white',
                'name': 'Spatial Gene Transcription',
            }),
    ]),
])

display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'ChcLkcYFQyKeXNObDS3-qA', 'tracks': {'top': [{'ty…

In [7]:
ts_density_eno1 = dfdensity(df, channel_col='gene', channel='ENO1', uuid='density_eno1')
ts_density_thbs1 = dfdensity(df, channel_col='gene', channel='THBS1', uuid='density_thbs1')

display, server, viewconf = higlass.display([
    View(
        x=0,
        y=0,
        width=4,
        height=12,
        tracks=[
            Track('left-axis'),
            Track('top-axis'),
            Track('heatmap',
                tileset=ts_density,
                position='center',
                height=400,
                options={
                    'colorRange': ['rgba(245,166,35,1.0)', 'rgba(208,2,27,1.0)', 'black'],
                    'backgroundColor': 'white',
                    'name': 'Spatial Gene Transcription',
                }),
        ]
    ),
    View(
        x=4,
        y=0,
        width=4,
        height=12,
        tracks=[
            Track('left-axis'),
            Track('top-axis'),
            Track('heatmap',
                tileset=ts_density_eno1,
                position='center',
                height=400,
                options={
                    'colorRange': ['rgba(128,217,255,1.0)', 'rgba(0,180,255,1.0)', 'rgba(0,90,128,1.0)', 'black'],
                    'backgroundColor': 'white',
                    'name': 'Spatial Gene Transcription of EN01',
                }
            ),
        ]
    ),
    View(
        x=8,
        y=0,
        width=4,
        height=12,
        tracks=[
            Track('left-axis'),
            Track('top-axis'),
            Track('heatmap',
                tileset=ts_density_thbs1,
                position='center',
                height=400,
                options={
                    'colorRange': ['rgba(128,217,255,1.0)', 'rgba(0,180,255,1.0)', 'rgba(0,90,128,1.0)', 'black'],
                    'backgroundColor': 'white',
                    'name': 'Spatial Gene Transcription of EN01',
                }
            ),
        ]
    ),
])

display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'AOtJXQu9RDCe53R_5SDZcw', 'tracks': {'top': [{'ty…